## Exploring Brazilian E-commerce Data

In this notebook, we are going to explore a dataset related to e-commerce sales in Brazil.

First, let's import the necessary Python libraries for data processing and visualization.

In [ ]:
import sys, json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import datetime as datetime

Next, we load a GeoJSON file that contains the map of Brazil's federal states.

In [ ]:
geojson = json.load(open('geojson/brasil_estados.json'))

We now load various CSV files into pandas dataframes. Each file represents different aspects of the orders:

1. `olist_customers_dataset.csv`: Customer data.
2. `olist_order_items_dataset.csv`: Order items data.
3. `olist_order_payments_dataset.csv`: Payment data.
4. `olist_order_reviews_dataset.csv`: Customer reviews data.
5. `olist_orders_dataset.csv`: Orders data.
6. `olist_products_dataset.csv`: Products data.
7. `product_category_name_translation.csv`: Product categories translation data.

In [ ]:
df0 = pd.read_csv('datasets/olist_customers_dataset.csv')
df2 = pd.read_csv('datasets/olist_order_items_dataset.csv')
df3 = pd.read_csv('datasets/olist_order_payments_dataset.csv')
df4 = pd.read_csv('datasets/olist_order_reviews_dataset.csv')
df5 = pd.read_csv('datasets/olist_orders_dataset.csv')
df6 = pd.read_csv('datasets/olist_products_dataset.csv')
df8 = pd.read_csv('datasets/product_category_name_translation.csv')

Now, we calculate and display the number of rows and columns in each dataframe to understand the size of our data.

In [ ]:
shapes = []
dfs = [df0,df2,df3,df4,df5,df6,df8]
for d in dfs :
    shapes.append(d.shape)

shapes = pd.DataFrame(data = shapes)# , columns = {'Rows','Coloumns'})
shapes

In [ ]:
print(df0.columns)
print(df2.columns)
print(df3.columns)
print(df4.columns)
print(df5.columns)
print(df6.columns)
print(df8.columns)

In [ ]:
df0 = df0[['customer_unique_id','customer_id','customer_zip_code_prefix','customer_city', 'customer_state']]
df2 = df2[['order_id', 'order_item_id', 'product_id', 'price', 'freight_value']]
df3 = df3[['order_id', 'payment_installments', 'payment_value']]
df4 = df4[['order_id','review_score']]
df5 = df5[['order_id', 'customer_id', 'order_purchase_timestamp','order_delivered_customer_date']]
df6 = df6[['product_id', 'product_category_name']]
df8 = df8[['product_category_name', 'product_category_name_english']]

In [ ]:
df = df0
del df0
df_orders = df2.merge(right=df3, on='order_id')
del df2,df3
df_orders = df_orders.merge(right=df4, on='order_id')
del df4
df_orders = df_orders.merge(right=df5, on='order_id')
del df5
df_orders = df_orders.merge(right=df6, on='product_id')
del df6
df_orders = df_orders.merge(right=df8, on='product_category_name')
del df8
df = df.merge(right=df_orders, on='customer_id')
del df_orders
df = df[['customer_unique_id','customer_city','customer_state','order_id','product_id',
         'price','payment_value','payment_installments','review_score','freight_value','order_purchase_timestamp','product_category_name_english']]
df.head()
df.shape

In [ ]:
df.isna().mean()

## Recency, frequency, order amount

### Order frequency by customer

In [ ]:
recence=df[['customer_unique_id','order_purchase_timestamp']]
recence.sort_values(by='order_purchase_timestamp', ascending=False)
recence.head(20)

In [ ]:
frequence=df[['customer_unique_id','order_purchase_timestamp']]

In [ ]:
frequence=frequence.groupby('customer_unique_id').count().sort_values('order_purchase_timestamp',ascending=False)
frequence=frequence.reset_index().rename(columns={"order_purchase_timestamp": "frequency"})

In [ ]:
df = df.merge(right=frequence, on='customer_unique_id')

### Last 20 orders

In [ ]:
df['order_purchase_timestamp'] = df['order_purchase_timestamp'].astype('datetime64[ns]')
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp'], format = '%YY%mm%dd')
df['since_last_purchase'] = (datetime.datetime.now() - df['order_purchase_timestamp'])
df['since_last_purchase']=df['since_last_purchase'].astype('timedelta64[D]')
df.sort_values('since_last_purchase').head(20)

### Order volume by customer and by state

#### By Customer

In [ ]:
commandes_client=df[['customer_unique_id', 'payment_value','customer_state','review_score']].groupby(by='customer_unique_id').agg({'payment_value': 'sum','review_score':'mean'}).reset_index()
commandes_client = commandes_client.sort_values('payment_value', ascending = False)
commandes_client.head(20)

In [ ]:
fig_bar = px.bar(commandes_client.head(20), y='payment_value', x='customer_unique_id',text='customer_unique_id',
             color= 'payment_value',color_continuous_scale = 'bluyl')          

fig_bar.update_traces(texttemplate='%{text:.2s %}', textposition='outside')
fig_bar.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig_bar.update_layout(xaxis={'categoryorder':'total ascending'})

fig_bar.show()

#### By state

In [ ]:
commandes_etat=df[['customer_state', 'payment_value','review_score']].groupby(by="customer_state").agg({'payment_value': 'sum','review_score':'mean'}).reset_index()
commandes_etat = commandes_etat.sort_values('payment_value', ascending = False)
commandes_etat.head(20)

In [ ]:
commandes_etat.head()

In [ ]:
fig_bar = px.bar(commandes_etat, y='payment_value', x='customer_state' ,text='customer_state',
             color='payment_value',color_continuous_scale = 'bluyl')          

fig_bar.update_traces(texttemplate='%{text:.2s %}', textposition='outside')
fig_bar.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig_bar.update_layout(xaxis={'categoryorder':'total ascending'})

fig_bar.show()

In [ ]:
fig_choropleth = px.choropleth(commandes_etat, geojson=geojson, locations='customer_state', color='review_score',
                           color_continuous_scale="bluyl",
                           scope='south america'
                          )
fig_choropleth.show()

In [ ]:
fig_choropleth = px.choropleth(commandes_etat, geojson=geojson, locations='customer_state', color='payment_value',
                           color_continuous_scale="bluyl",
                           scope='south america'
                          )
fig_choropleth.show()

### Stars

He we define "Stars" as a combination. First customers are ordered by payment value, and then by frequency. The last order must be recent (at least january 2018). Thus the most important criterion after the recency is the total amount and then the frequency of purchases. 

In [ ]:
stars=df[['customer_unique_id', 'payment_value','frequency','since_last_purchase']]
#stars=stars[stars['since_last_purchase'] < 1770] #depuis janvier 2018
stars=stars.groupby(by="customer_unique_id").agg({'payment_value':'sum'}).reset_index()
stars = stars.merge(right=frequence, on='customer_unique_id')
stars = stars.sort_values(['payment_value','frequency'], ascending = [False,False])
stars['average_purchase_value']=np.round(stars['payment_value']/stars['frequency'],2)
stars.head(5)


## Preparation of the data for clustering

As the clustering algorithm only works with numbers we transform each individual value in the table as a "category code" same values thus have same numerical values thus can be used in the clustering. We do the same for datetimes. 

In [ ]:
df['customer_unique_id'] = pd.Categorical(df['customer_unique_id'])
df['customer_code'] = df['customer_unique_id'].cat.codes

df['customer_city'] = pd.Categorical(df['customer_city'])
df['city_code'] = df['customer_city'].cat.codes

df['customer_state'] = pd.Categorical(df['customer_state'])
df['state_code'] = df['customer_state'].cat.codes

df['order_id'] = pd.Categorical(df['order_id'])
df['order_code'] = df['order_id'].cat.codes

df['product_id'] = pd.Categorical(df['product_id'])
df['product_code'] = df['product_id'].cat.codes

df['product_id'] = pd.Categorical(df['product_id'])
df['product_code'] = df['product_id'].cat.codes

df['order_purchase_timestamp_str'] = df['order_purchase_timestamp']
df['order_purchase_timestamp'] = pd.to_datetime(df['order_purchase_timestamp']).astype(np.int64)

df['product_category_name_english'] = pd.Categorical(df['product_category_name_english'])
df['product_category_code'] = df['product_category_name_english'].cat.codes

df.head()

We also check for correlations, having features with too much correlation will negatively influence the performance of the clustering.

In [ ]:
sns.set(rc = {'figure.figsize':(30,15)})
sns.heatmap(df.corr(), annot=True, cmap='YlGnBu')

In [ ]:
df_final=df[['order_purchase_timestamp_str','order_purchase_timestamp','customer_code',
    'price','payment_value','review_score','freight_value','frequency','since_last_purchase']]

We now export the data to be processed by the following notebook

In [ ]:
del df

In [ ]:
df_final.to_csv('datasets/cleaned.csv', index = False, header=True)